In [1]:
#import dependencies
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re
import pandas as pd
from splinter import Browser 

In [2]:
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path)

In [3]:
#set the url and visit it by sprinter
url = 'https://developer.mapquest.com/documentation/tools/latitude-longitude-finder/'
browser.visit(url)

In [4]:
# read csv file as dataframe
nyc_bookstores_df = pd.read_csv('nyc_bookstores.csv')
nyc_bookstores_df.head()

,Name,Address,Link
0,Berl's Brooklyn Poetry Shop,"141 Front St., Brooklyn, NY 11201",http://www.berlspoetry.com/
1,Bookmark Shoppe,"8415 3rd Ave, Brooklyn, NY 11209",https://www.bookmarkshoppe.com/
2,Books Are Magic,"225 Smith St., Brooklyn, NY 11231",https://www.booksaremagic.net/
3,Cafe Con Libros,"724 Prospect Pl. Brooklyn, NY 11216",https://www.cafeconlibrosbk.com/
4,Catland Books,"987 Flushing Ave, Brooklyn, NY 11206",https://www.catlandbooks.com/
...,...,...,...
63,Kew & Willow Books,"81-63 Lefferts Blvd, Kew Gardens\tNY, 11415",https://www.kewandwillow.com/
64,Topos Bookstore Cafe,"788 Woodward Ave., Ridgewood, NY 11385",https://www.toposbookstore.com/
65,Turn the Page Again!,"39-15 Bell Blvd, Bayside, NY 11361",http://www.turnthepageagain.com/
66,Everything Goes Book Cafe,"208 Bay St., Staten Island, NY 10301",http://www.etgstores.com/bookcafe/


In [5]:
# Temp manual fixes to address column
nyc_bookstores_df['Address'][29] = '151 East Broadway, New York, NY,10002'
nyc_bookstores_df['Address'][59] = '238 W 10th St., New York, NY 10014'

In [21]:
# Get lat-lng results using address column
find_button = browser.find_by_id('btnGC')
clear_button = browser.find_by_id('clearDiv')
results = []
for address in nyc_bookstores_df['Address']:
    browser.find_by_name('street').type(address)
    find_button.click()
    result = browser.find_by_id('resultDiv').text
    results.append(result)
    time.sleep(1.5)
    clear_button.click()

In [23]:
# Clean up results and get latitudes and longtitudes seperately
latitudes= []
longitudes = []
for item in results:
    result_list = item.split(',')
    lat_result = result_list[0].split(':')
    lng_result = result_list[1].split(':')
    latitudes.append(float(lat_result[1]))
    longitudes.append(float(lng_result[1]))
    

In [27]:
# Add latitude and Lontitude to the df
nyc_bookstores_df['Latitudes'] = latitudes
nyc_bookstores_df['Longitudes'] = longitudes

In [28]:
# Save dataframe as csv
nyc_bookstores_df.to_csv('nyc_bookstores_location.csv',index=False)

In [29]:
# Close browser
browser.quit()